In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer

/home/aastroza/miniconda3/envs/ids-gpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

In [4]:
class Model:
    def init_model(self):
        self.model = SentenceTransformer(
            EMBEDDING_MODEL_NAME,
        )

    def __enter__(self):
        self.init_model()

    def __exit__(self, *args):
        pass

    def uncached_embed(self, texts: list[str]) -> list[str]:
        if not hasattr(self, "model"):
            self.init_model()
        embeddings = self.model.encode(texts, show_progress_bar=True)
        return embeddings

In [5]:
model = Model()

In [11]:
df = pd.read_csv('data/raw/constitucion_vigente_corregida.csv')

In [17]:
pred = model.uncached_embed(list(df['texto']))

Batches: 100%|██████████| 11/11 [00:00<00:00, 24.83it/s]


In [22]:
df['emb'] = list(pred)

In [24]:
df.to_csv('data/processed/constitucion_vigente_embeddings.csv')